# Experiment 007: Verify Long C++ Optimization Results

In [1]:
import numpy as np
import pandas as pd
from numba import njit
from numba.typed import List as NumbaList
import math

# Tree constants
TRUNK_W = 0.15
TRUNK_H = 0.2
BASE_W = 0.7
MID_W = 0.4
TOP_W = 0.25
TIP_Y = 0.8
TIER_1_Y = 0.5
TIER_2_Y = 0.25
BASE_Y = 0.0
TRUNK_BOTTOM_Y = -TRUNK_H

In [2]:
@njit(cache=True)
def get_tree_vertices(cx, cy, angle_deg):
    angle_rad = angle_deg * math.pi / 180.0
    cos_a, sin_a = math.cos(angle_rad), math.sin(angle_rad)
    vertices = np.empty((15, 2), dtype=np.float64)
    pts = np.array([[0.0, TIP_Y], [TOP_W/2, TIER_1_Y], [TOP_W/4, TIER_1_Y],
                    [MID_W/2, TIER_2_Y], [MID_W/4, TIER_2_Y], [BASE_W/2, BASE_Y],
                    [TRUNK_W/2, BASE_Y], [TRUNK_W/2, TRUNK_BOTTOM_Y],
                    [-TRUNK_W/2, TRUNK_BOTTOM_Y], [-TRUNK_W/2, BASE_Y],
                    [-BASE_W/2, BASE_Y], [-MID_W/4, TIER_2_Y], [-MID_W/2, TIER_2_Y],
                    [-TOP_W/4, TIER_1_Y], [-TOP_W/2, TIER_1_Y]], dtype=np.float64)
    for i in range(15):
        rx = pts[i,0] * cos_a - pts[i,1] * sin_a
        ry = pts[i,0] * sin_a + pts[i,1] * cos_a
        vertices[i,0], vertices[i,1] = rx + cx, ry + cy
    return vertices

@njit(cache=True)
def polygon_bounds(vertices):
    min_x, min_y = vertices[0,0], vertices[0,1]
    max_x, max_y = vertices[0,0], vertices[0,1]
    for i in range(1, vertices.shape[0]):
        x, y = vertices[i,0], vertices[i,1]
        if x < min_x: min_x = x
        if x > max_x: max_x = x
        if y < min_y: min_y = y
        if y > max_y: max_y = y
    return min_x, min_y, max_x, max_y

@njit(cache=True)
def get_side_length(all_vertices):
    min_x, min_y, max_x, max_y = math.inf, math.inf, -math.inf, -math.inf
    for verts in all_vertices:
        x1, y1, x2, y2 = polygon_bounds(verts)
        if x1 < min_x: min_x = x1
        if y1 < min_y: min_y = y1
        if x2 > max_x: max_x = x2
        if y2 > max_y: max_y = y2
    return max(max_x - min_x, max_y - min_y)

@njit(cache=True)
def calculate_score_numba(all_vertices):
    side = get_side_length(all_vertices)
    return side * side / len(all_vertices)

In [3]:
def load_submission_data(filepath):
    df = pd.read_csv(filepath)
    all_xs, all_ys, all_degs = [], [], []
    for n in range(1, 201):
        group = df[df["id"].str.startswith(f"{n:03d}_")].sort_values("id")
        for _, row in group.iterrows():
            all_xs.append(float(str(row["x"]).replace('s', '')))
            all_ys.append(float(str(row["y"]).replace('s', '')))
            all_degs.append(float(str(row["deg"]).replace('s', '')))
    return np.array(all_xs), np.array(all_ys), np.array(all_degs)

def calculate_total_score(all_xs, all_ys, all_degs):
    total, idx = 0.0, 0
    for n in range(1, 201):
        vertices = NumbaList()
        for i in range(n): vertices.append(get_tree_vertices(all_xs[idx+i], all_ys[idx+i], all_degs[idx+i]))
        total += calculate_score_numba(vertices)
        idx += n
    return total

In [4]:
# Load and compare
print("Loading baseline...")
baseline_xs, baseline_ys, baseline_degs = load_submission_data('/home/code/santa-2025-csv/santa-2025.csv')
baseline_score = calculate_total_score(baseline_xs, baseline_ys, baseline_degs)
print(f"Baseline score: {baseline_score:.6f}")

print("\nLoading C++ optimized result...")
optimized_xs, optimized_ys, optimized_degs = load_submission_data('/home/code/submission_v18.csv')
optimized_score = calculate_total_score(optimized_xs, optimized_ys, optimized_degs)
print(f"Optimized score: {optimized_score:.6f}")

print(f"\nImprovement: {baseline_score - optimized_score:.6f}")
print(f"Target: 68.922808")
print(f"Gap to target: {optimized_score - 68.922808:.6f}")

Loading baseline...


Baseline score: 70.676102

Loading C++ optimized result...


Optimized score: 70.676102

Improvement: 0.000000
Target: 68.922808
Gap to target: 1.753294


In [5]:
# Save the better solution
if optimized_score < baseline_score:
    import shutil
    shutil.copy('/home/code/submission_v18.csv', '/home/submission/submission.csv')
    shutil.copy('/home/code/submission_v18.csv', '/home/code/experiments/007_long_cpp/submission.csv')
    print("Saved optimized submission!")
    final_score = optimized_score
else:
    import shutil
    shutil.copy('/home/code/santa-2025-csv/santa-2025.csv', '/home/submission/submission.csv')
    shutil.copy('/home/code/santa-2025-csv/santa-2025.csv', '/home/code/experiments/007_long_cpp/submission.csv')
    print("No improvement - saved baseline")
    final_score = baseline_score

Saved optimized submission!


In [6]:
# Save metrics
import json
metrics = {
    'baseline_score': baseline_score,
    'optimized_score': optimized_score,
    'final_score': final_score,
    'improvement': baseline_score - optimized_score,
    'target': 68.922808,
    'optimizer': 'tree_packer_v18',
    'iterations': 1000000,
    'restarts': 100
}
with open('/home/code/experiments/007_long_cpp/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)
print("Metrics saved")

Metrics saved
